* ISIC Challenge datasets

In [1]:
!git clone https://github.com/andreiionut1411/trustworthy-federated-skin-lesion.git

Cloning into 'trustworthy-federated-skin-lesion'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 36 (delta 7), reused 32 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 2.31 MiB | 9.95 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!pip install -U "flwr[simulation]"
!pip install -U ray

INFO: pip is looking at multiple versions of grpcio-health-checking to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-health-checking to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.31.0
    Uninstalling ray-2.31.0:
      Successfully uninstalled ray-2.31.0


In [3]:
!pip install -r /content/trustworthy-federated-skin-lesion/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/AITDM/train.zip -d /content

Datele de ieșire de afișat au fost trunchiate la ultimele 5000 linii.
  inflating: /content/train/ISIC_0084086.jpg  
  inflating: /content/train/ISIC_7843406.jpg  
  inflating: /content/train/ISIC_4198374.jpg  
  inflating: /content/train/ISIC_1728564.jpg  
  inflating: /content/train/ISIC_1699154.jpg  
  inflating: /content/train/ISIC_6882841.jpg  
  inflating: /content/train/ISIC_6481862.jpg  
  inflating: /content/train/ISIC_3908914.jpg  
  inflating: /content/train/ISIC_4711419.jpg  
  inflating: /content/train/ISIC_1725997.jpg  
  inflating: /content/train/ISIC_6482577.jpg  
  inflating: /content/train/ISIC_0927719.jpg  
  inflating: /content/train/ISIC_5359062.jpg  
  inflating: /content/train/ISIC_3563586.jpg  
  inflating: /content/train/ISIC_0156642.jpg  
  inflating: /content/train/ISIC_9249257.jpg  
  inflating: /content/train/ISIC_4266455.jpg  
  inflating: /content/train/ISIC_2286273.jpg  
  inflating: /content/train/ISIC_0650784.jpg  
  inflating: /content/train/ISIC_8943

1. Import the libs

In [3]:
import pandas as pd
import numpy as np
import threading
from collections import defaultdict
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import flwr as fl
from PIL import Image
from sklearn.model_selection import train_test_split
import os
from torchvision import transforms
from tqdm import tqdm
from flwr.common import Context

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


2. Statistics of the datasets

All the images were preprocessed using the `data_processing/preprocess.py` to images of 256 x 256 pixels for uniformity to reduce the domain shift.

ISIC 2020 - The dataset we will use for federated learning

The dataset has only 2 labels for the images: malignant and benign.

In [ ]:
df = pd.read_csv("labels/ISIC_2020_Training_GroundTruth.csv")
num_images = len(df)
num_patients = df['patient_id'].nunique()
num_classes = df['benign_malignant'].nunique()
class_counts = df['benign_malignant'].value_counts()

print(f"Total images: {num_images}")
print(f"Total patients: {num_patients}")
print(f"Classes: {num_classes}")
print("Class distribution:\n", class_counts)
print("\nAge statistics:")
print(df['age_approx'].describe())
print("\nSex distribution:\n", df['sex'].value_counts())
print("\nAnatomical site distribution:\n", df['anatom_site_general_challenge'].value_counts())

Total images: 33126
Total patients: 2056
Classes: 2
Class distribution:
 benign_malignant
benign       32542
malignant      584
Name: count, dtype: int64

Age statistics:
count    33058.000000
mean        48.870016
std         14.380360
min          0.000000
25%         40.000000
50%         50.000000
75%         60.000000
max         90.000000
Name: age_approx, dtype: float64

Sex distribution:
 sex
male      17080
female    15981
Name: count, dtype: int64

Anatomical site distribution:
 anatom_site_general_challenge
torso              16845
lower extremity     8417
upper extremity     4983
head/neck           1855
palms/soles          375
oral/genital         124
Name: count, dtype: int64


3. Split the data for Federated Learning

We need to simulate the real life where hospitals have different types of pacients (different ages, different social backgrounds, etc). We decided to split the data into 4 clients. Each client will receive a different data split. All the data for a particular patient is inside the same client (hospital) - people tend to go to the hospital they are used to.

Each hospital has a different "story" to try to mimic the real world:

1) Hospital A - Big city hospital with most of the patients (45% of the total number of patients). Also, the remaining patients that had missing metadata is assigned to this hospital as well because there can be some mistakes at bigger hospitals with many patients.
2) Hospital B - Hospital focused more on younger people (all the patients younger than 40 years, max 20% of the total number)
3) Hospital C - Hospital focused on the upper body. It contains all the patients that come to take images of lesions of "head/neck", "oral/genital", "upper extremity"
4) Hospital D - Hospital with the rest. Can be a regular dermathological hospital in a city.

In [ ]:
def split_patients_federated(csv_path, seed=42):
    np.random.seed(seed)
    df = pd.read_csv(csv_path)

    # Unique patients
    patients = df.groupby("patient_id").agg({
        "age_approx": "mean",
        "anatom_site_general_challenge": lambda x: x.mode().iloc[0] if not x.mode().empty else "unknown",
        "benign_malignant": lambda x: "malignant" if "malignant" in x.values else "benign"
    }).reset_index()

    # ---- Define hospital groups ----
    # Hospital A: Urban General (largest)
    # Hospital B: Youth Dermatology (<40)
    # Hospital C: Head/Neck specialist
    # Hospital D: Rural/Older

    num_patients = len(patients)
    hospital_a_size = int(0.45 * num_patients)
    hospital_b_size = int(0.20 * num_patients)
    hospital_c_size = int(0.15 * num_patients)

   # Hospital B -> Young patients (<40)
    available_patients = patients.copy()
    young = available_patients[available_patients["age_approx"] < 40]
    if len(young) > hospital_b_size:
        young = young.sample(hospital_b_size, random_state=seed)
    hospital_b_patients = set(young["patient_id"])

    # Hospital C -> Upper body focus
    available_patients = patients[~patients["patient_id"].isin(hospital_b_patients)]
    head_sites = ["head/neck", "oral/genital", "upper extremity"]
    upper_body_patients = available_patients[available_patients["anatom_site_general_challenge"].isin(head_sites)]
    if len(upper_body_patients) > hospital_c_size:
        upper_body_patients = upper_body_patients.sample(hospital_c_size, random_state=seed)
    hospital_c_patients = set(upper_body_patients["patient_id"])

    # Hospital A -> Big City Hospital
    available_patients = patients[~patients["patient_id"].isin(hospital_b_patients | hospital_c_patients)]
    hospital_a_patients = set(
        available_patients.sample(hospital_a_size, random_state=seed)["patient_id"]
    )

    # Hospital D -> Rest
    hospital_d_patients = set(patients["patient_id"]) - (hospital_a_patients | hospital_b_patients | hospital_c_patients)

    hospitals = {
        "Hospital_A_Urban": df[df["patient_id"].isin(hospital_a_patients)],
        "Hospital_B_Youth": df[df["patient_id"].isin(hospital_b_patients)],
        "Hospital_C_Upper_Body": df[df["patient_id"].isin(hospital_c_patients)],
        "Hospital_D_Rural": df[df["patient_id"].isin(hospital_d_patients)]
    }

    df["hospital_id"] = 0
    df.loc[df["patient_id"].isin(hospital_a_patients), "hospital_id"] = 1
    df.loc[df["patient_id"].isin(hospital_b_patients), "hospital_id"] = 2
    df.loc[df["patient_id"].isin(hospital_c_patients), "hospital_id"] = 3
    df.loc[df["patient_id"].isin(hospital_d_patients), "hospital_id"] = 4
    df.to_csv(csv_path, index=False)

    # --- Print summaries ---
    print("\n=== Hospital Summary ===")
    for name, hdf in hospitals.items():
        counts = hdf["benign_malignant"].value_counts()
        mean_age = hdf["age_approx"].mean()
        top_site = hdf["anatom_site_general_challenge"].mode().iloc[0]
        print(f"\n{name}:")
        print(f" Patients: {hdf['patient_id'].nunique():5d} | Images: {len(hdf):5d}")
        print(f" Avg. age: {mean_age:.1f} | Dominant body part: {top_site}")
        print(f" Benign: {counts.get('benign',0):5d} | Malignant: {counts.get('malignant',0):4d}")

    return hospitals

In [ ]:
csv_path = "labels/ISIC_2020_Training_GroundTruth.csv"
seed = 42
client_splits = split_patients_federated(csv_path, seed)


=== Hospital Summary ===

Hospital_A_Urban:
 Patients:   925 | Images: 14505
 Avg. age: 54.8 | Dominant body part: torso
 Benign: 14212 | Malignant:  293

Hospital_B_Youth:
 Patients:   411 | Images:  8141
 Avg. age: 31.1 | Dominant body part: torso
 Benign:  8074 | Malignant:   67

Hospital_C_Upper_Body:
 Patients:   163 | Images:  1429
 Avg. age: 58.2 | Dominant body part: upper extremity
 Benign:  1367 | Malignant:   62

Hospital_D_Rural:
 Patients:   557 | Images:  9051
 Avg. age: 54.0 | Dominant body part: torso
 Benign:  8889 | Malignant:  162


4. Write the model to train

In [4]:
class SmallCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.pool = nn.AdaptiveAvgPool2d((4,4))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*4*4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

5. Create the Federated Learning set-up

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
class SkinLesionDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row['image_name'] + ".jpg")
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
        label = 0 if row['benign_malignant'] == 'benign' else 1
        return img, label

In [11]:
def train_local(model, dataloader, epochs, lr, device):
    model.train()
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        # Only one tqdm bar for the batches in this epoch
        pbar = tqdm(dataloader, desc=f"Train epoch {epoch+1}/{epochs}", leave=False, ncols=100)
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X.size(0)
            # Update postfix with current batch loss
            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        # Optionally, show average epoch loss after epoch finishes
        print(f"Epoch {epoch+1}/{epochs} - Avg loss: {running_loss/len(dataloader.dataset):.4f}")

    return running_loss / len(dataloader.dataset)


def evaluate(model, dataloader, device):
    model.eval()
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    total_loss, correct, total = 0.0, 0, 0

    pbar = tqdm(dataloader, desc="Eval", leave=False, ncols=100)
    with torch.no_grad():
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            loss = criterion(outputs, y)
            total_loss += loss.item() * X.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += X.size(0)
            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    return total_loss / total, correct / total


In [ ]:
def load_partition(csv_path, hospital_id, img_dir, test_size=0.2):
    df = pd.read_csv(csv_path)
    part_df = df[df['hospital_id'] == hospital_id]
    train_df, val_df = train_test_split(part_df, test_size=test_size, stratify=part_df['benign_malignant'], random_state=42)
    transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    return SkinLesionDataset(train_df, img_dir, transform), SkinLesionDataset(val_df, img_dir, transform)


In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_ds, val_ds, device, local_epochs=1, lr=1e-4):
        self.model = model
        self.trainloader = DataLoader(train_ds, batch_size=16, shuffle=True)
        self.testloader = DataLoader(val_ds, batch_size=16)
        self.device = device
        self.local_epochs = local_epochs
        self.lr = lr

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        state_dict = {k: torch.tensor(v) for k,v in zip(self.model.state_dict().keys(), parameters)}
        self.model.load_state_dict(state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train_local(self.model, self.trainloader, epochs=self.local_epochs, lr=self.lr, device=self.device)
        return self.get_parameters({}), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, acc = evaluate(self.model, self.testloader, self.device)
        return float(loss), len(self.testloader.dataset), {"accuracy": float(acc)}

In [15]:
csv_path = "/content/trustworthy-federated-skin-lesion/labels/ISIC_2020_Training_GroundTruth.csv"
img_dir = "/content/train/"
df = pd.read_csv(csv_path)
hospital_ids = sorted(df['hospital_id'].unique())  # e.g., [1,2,3,4]

from concurrent.futures import ThreadPoolExecutor
import flwr as fl

# Proper client function for Flower >= 1.2
from flwr.common import Context


def client_fn(cid: str):
    # cid is a string like "0", "1", "2", "3"
    hid = hospital_ids[int(cid)]  # each client gets a hospital partition
    train_ds, val_ds = load_partition(csv_path, hid, img_dir)
    model = SmallCNN()
    return FlowerClient(model, train_ds, val_ds, device, local_epochs=1, lr=1e-4)

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=len(hospital_ids),
    min_available_clients=len(hospital_ids),
)

client_resources = {"num_cpus": 1, "num_gpus": 1}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=len(hospital_ids),
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
    client_resources=client_resources,  # <-- allocate GPU here
)

Train epoch 1/1:  87%|████████▋ | 392/453 [00:33<00:04, 12.83it/s, loss=0.0254]
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is depr

(ClientAppActor pid=8600) Epoch 1/1 - Avg loss: 0.0911


(ClientAppActor pid=8600) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=8600) 
(ClientAppActor pid=8600)             This is a deprecated feature. It will be removed
(ClientAppActor pid=8600)             entirely in future versions of Flower.
(ClientAppActor pid=8600)         
(ClientAppActor pid=8600) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Train epoch 1/1: 100%|██████████████████████████████▉| 724/726 [00:59<00:00, 10.78it/s, loss=0.0357]
(ClientAppActor pid=8600) 
(ClientAppActor pid=8600)             This is a deprecated feature. It will be removed
(ClientAppActor pid=8600)             entirel

(ClientAppActor pid=8600) Epoch 1/1 - Avg loss: 0.0973


(ClientAppActor pid=8600) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Train epoch 1/1:  97%|████████████████████████████████ | 70/72 [00:05<00:00, 13.02it/s, loss=0.0393]
                                                                                                    


...

(ClientAppActor pid=8600) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Train epoch 1/1: 100%|██████████████████████████████▉| 452/453 [00:37<00:00, 12.59it/s, loss=0.0027]
INFO :      aggregate_fit: received 4 results and 0 failures
INFO :      configure_evaluate: strategy sampled 4 clients (out of 4)
                                                                                                    
(ClientAppActor pid=8600) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=8600) 
(ClientAppActor pid=8600)             This is a deprecated feature. It will be removed
(ClientAppActor pid=8600)             e

(ClientAppActor pid=8600) Epoch 1/1 - Avg loss: 0.0818


Eval:  83%|████████████████████████████████████▋       | 15/18 [00:00<00:00, 20.19it/s, loss=0.2806]
(ClientAppActor pid=8600) 
(ClientAppActor pid=8600)             This is a deprecated feature. It will be removed
(ClientAppActor pid=8600)             entirely in future versions of Flower.
(ClientAppActor pid=8600)         
(ClientAppActor pid=8600) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Eval:  97%|█████████████████████████████████████████▋ | 99/102 [00:06<00:00, 17.43it/s, loss=0.0628]
(ClientAppActor pid=8600) 
(ClientAppActor pid=8600)             This is a deprecated feature. It will be removed
(ClientAppActor pid=8600)             entirely in future versions of Flower.
(ClientAppActor pid=8600)         
(ClientAppActor pid=8600) WARNING :   Deprecation Warning: The `client_fn` function must return an instance

History (loss, distributed):
	round 1: 0.13197390997555528
	round 2: 0.09961162840938234
	round 3: 0.09641094577796808

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
